## Goal - Merge several shapefiles using OGR2OGR
##      - Project the result to PT-TM06/ETRS89

In [ ]:
import os
from osgeo import ogr
from pycode.geofiles import drv_name, list_esri_shp
from pycode.oss import execmd

### Paths to data

In [ ]:
datafolder = '/mnt/d/mystuff/pwgis/mod1_ex2/'
outfile    = '/mnt/d/mystuff/pwgis/mod1_ex2/results/altimerge_ogr.shp'

## List all files in data folder

In [ ]:
shps = list_esri_shp(datafolder)

In [ ]:
shps

### Merge all shapefiles into one

In [ ]:
tmpshp = os.path.join(
    os.path.dirname(outfile),
    f'tmp_{os.path.basename(outfile)}'
)

out_drv = drv_name(tmpshp)

for i in range(len(shps)):
    if not i:
        # Create output and copy some features of one layer (first in shps)
        cmd = 'ogr2ogr -f "{}" {} {}'.format(
            out_drv, tmpshp, os.path.join(datafolder, shps[i])
        )
    
    else:
        cmd = 'ogr2ogr -f "{}" -update -append {} {}'.format(
            out_drv, tmpshp, os.path.join(datafolder, shps[i])
        )
        
    cmdout = execmd(cmd)

### Project to ETRS89

In [ ]:
data = ogr.GetDriverByName(drv_name(tmpshp)).Open(tmpshp)

lyr   = data.GetLayer() 
spref = lyr.GetSpatialRef()

epsg = int(str(spref.GetAttrValue('AUTHORITY', 1)))
print(epsg)

cmd = 'ogr2ogr -f "{}" {} {} -s_srs EPSG:{} -t_srs EPSG:{}'.format(
    drv_name(outfile), outfile, tmpshp,
    str(epsg), str(3763)
)

outcmd = execmd(cmd)